In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('fivethirtyeight')

# Задание №1

In [44]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [160]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [46]:
housing.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [49]:
data = housing.data

In [50]:
feature_names = housing.feature_names

In [51]:
target  = housing.target

In [52]:
X = pd.DataFrame(data, columns=feature_names)
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [53]:
Y = pd.DataFrame(target, columns=["price"])
Y.head()

,price
0,4.526
1,3.585
2,3.521
3,3.413
4,3.422


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [62]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [63]:
pred = lr.predict(X_test)
check = pd.DataFrame({
    'Тестовые данные': y_test['price'], 
    'Предсказание': pred.flatten()})

check.head(10)

,Тестовые данные,Предсказание
20046,0.47700,0.726049
3024,0.45800,1.767434
15663,5.00001,2.710922
20484,2.18600,2.835147
9814,2.78000,2.606958
13311,1.58700,2.010739
7113,1.98200,2.640674
7668,1.57500,2.167062
18246,3.40000,2.740121
5723,4.46600,3.903615


В заданиии нет но мне интересно оценить точность

In [64]:
mean_squared_error(check['Предсказание'], check['Тестовые данные'])

0.5305677824766757

In [65]:
r2_score( check['Тестовые данные'], check['Предсказание'])

0.595770232606166

In [66]:
mean_absolute_error(check['Тестовые данные'], check['Предсказание'])

0.5272474538305951

# Задание 2

In [40]:
from sklearn.ensemble import RandomForestRegressor

In [41]:
model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

In [77]:
y_train.values[:, 0]

array([1.938, 1.697, 2.598, ..., 2.221, 2.835, 3.25 ])

In [68]:
model.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(max_depth=12, n_estimators=1000, random_state=42)

In [85]:
model_pred = model.predict(X_test)
check_model = pd.DataFrame({
    'model тест. данные': y_test['price'], 
    'Предсказание': model_pred.flatten()})

In [83]:
check_model.head(10)

,model тест. данные,Пердсказание
20046,0.47700,0.528463
3024,0.45800,0.805791
15663,5.00001,4.827227
20484,2.18600,2.511012
9814,2.78000,2.210708
13311,1.58700,1.735657
7113,1.98200,2.385568
7668,1.57500,1.665209
18246,3.40000,2.675503
5723,4.46600,4.843141


In [86]:
r2_score( check_model['model тест. данные'], check_model['Предсказание'])

0.7945055702215934

In [87]:
r2_score( check['Тестовые данные'], check['Предсказание'])

0.595770232606166

Модель с использованием RandomForestRegressor показала результат на треть лучше, но затратила значительно больше времени.

# Задание №3

In [90]:
?RandomForestRegressor

In [91]:
model.feature_importances_

array([0.55926878, 0.05043086, 0.03984255, 0.02325535, 0.02430738,
       0.13912464, 0.08153759, 0.08223285])

In [97]:
model.feature_importances_.sum()

1.0

In [105]:
model_feature_importances = pd.DataFrame({
    'Вес': model.feature_importances_, 
    'Наименование': feature_names})

In [106]:
model_feature_importances.sort_values(by='Вес', ascending=False)

,Вес,Наименование
0,0.559269,MedInc
5,0.139125,AveOccup
7,0.082233,Longitude
6,0.081538,Latitude
1,0.050431,HouseAge
2,0.039843,AveRooms
4,0.024307,Population
3,0.023255,AveBedrms


In [107]:
model_feature_importances.nlargest(2, 'Вес')

,Вес,Наименование
0,0.559269,MedInc
5,0.139125,AveOccup


# Задание 4

In [116]:
df = pd.read_csv('creditcard.csv')
pd.options.display.max_columns = 100

In [113]:
df['Class'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [118]:
df.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,1.341262,0.359894,-0.358091,-0.137134,0.517617,0.401726,-0.058133,0.068653,-0.033194,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,-1.416907,-0.153826,-0.751063,0.167372,0.050144,-0.443587,0.002821,-0.611987,-0.045575,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,-0.619468,0.291474,1.757964,-1.323865,0.686133,-0.076127,-1.222127,-0.358222,0.324505,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,-0.705117,-0.110452,-0.286254,0.074355,-0.328783,-0.210077,-0.499768,0.118765,0.570328,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,1.017614,0.836390,1.006844,-0.443523,0.150219,0.739453,-0.540980,0.476677,0.451773,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [119]:
X = df.drop('Class', axis=1)

In [122]:
y = df['Class']

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

In [146]:
print(
f'X_train : {X_train.shape}\n\
X_test : {X_test.shape}\n\
y_train : {y_train.shape}\n\
y_test : {y_test.shape}\n')

X_train : (199364, 30)
X_test : (85443, 30)
y_train : (199364,)
y_test : (85443,)



In [138]:
parameters = [{'n_estimators': [10, 15],
'max_features': np.arange(3, 5),
'max_depth': np.arange(4, 7)}]

In [ ]:
?GridSearchCV 

In [141]:
model_gscv = GridSearchCV(
    estimator=RandomForestClassifier(random_state=100),
    param_grid=parameters,
    scoring='roc_auc',
    cv=3)

In [142]:
model_gscv.fit(X_train, y_train)


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=100),
             param_grid=[{'max_depth': array([4, 5, 6]),
                          'max_features': array([3, 4]),
                          'n_estimators': [10, 15]}],
             scoring='roc_auc')

In [143]:
model_gscv.best_params_

{'max_depth': 6, 'max_features': 3, 'n_estimators': 15}

In [144]:
rfc = RandomForestClassifier(max_depth=6, max_features=3, n_estimators=15)

In [145]:
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, max_features=3, n_estimators=15)

In [148]:
rfc_pred = rfc.predict_proba(X_test)

In [154]:
rfc_pred

array([[9.99262609e-01, 7.37390791e-04],
       [9.99652193e-01, 3.47807172e-04],
       [9.99673313e-01, 3.26686655e-04],
       ...,
       [9.99591374e-01, 4.08625653e-04],
       [9.99245337e-01, 7.54662892e-04],
       [9.94247750e-01, 5.75224961e-03]])

In [158]:
y_pred_proba = rfc_pred[:, 1]

In [159]:
y_pred_proba.shape

(85443,)

In [162]:
roc_auc_score(y_test, y_pred_proba)

0.9484502117452465